In [ ]:
# Basic setup
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import edgel_utils as edgel

In [ ]:
import pandas as pd
FILE_PATH = "./test_files/completion_test.jsonl"
data = pd.read_json(FILE_PATH, lines=True)

from edgel_utils import get_prompt_graphs, parse_edge, V_JSON
prompt_graphs = get_prompt_graphs(data.iloc[1]['prompt'])
TEST_GRAPH = prompt_graphs[1] # select the last graph in the prompt

lines = TEST_GRAPH.split('\n')
first_edge = lines[1] # 0 is header

is_edge, src_id, tgt_id, edge_label, src_node_label, tgt_node_label = parse_edge(first_edge, version=V_JSON)

TEST_NODE = src_node_label
print(TEST_GRAPH)
print(TEST_NODE)

In [ ]:
correct, graph = edgel.parse_graph(TEST_GRAPH, directed=True, version=edgel.V_JSON, parse_labels_json=True, reduce_labels=True)
assert correct is True

In [ ]:
node_obj = edgel.ChangeGraphNode.from_string(TEST_NODE)

In [ ]:
reduced_string = edgel.ChangeGraphNode.to_json(TEST_NODE, reduce=True)

In [ ]:
reduced_string

In [ ]:
from mining.plot_utils import plot_graphs

plot_graphs([graph], json_labels=True)

# Determining anchor nodes of completion


Load file:

In [ ]:
import pandas as pd
FILE_PATH = "./test_files/completion_test.jsonl"
data = pd.read_json(FILE_PATH, lines=True)


Read out prompt and completion:

In [ ]:

from edgel_utils import get_prompt_graphs

example_id = 0 # id to choose from the examples

example = data.iloc[example_id]

def get_last_graph(example):
    

    prompt_graphs = get_prompt_graphs(example['prompt'])
    prompt = prompt_graphs[-1] # select the last graph in the prompt

    completion =  'e' + example['completion']

    full_graph = prompt + "\n" + completion
    return prompt, completion, full_graph

prompt, completion, full_graph = get_last_graph(example)


Determine anchor nodes.

In [ ]:
from edgel_utils import get_anchor_node_ids
anchor_ids = get_anchor_node_ids(prompt, completion)

In [ ]:

correct, graph = edgel.parse_graph(full_graph, directed=True, version=edgel.V_JSON, parse_labels_json=True, reduce_labels=True, serialized_ids=anchor_ids)
assert correct is True

In [ ]:
anchor_ids

In [ ]:
from mining.plot_utils import plot_graphs

plot_graphs([graph], json_labels=True)

Now, we check the functionality to get only the "anchored" completion graph:

In [ ]:
from edgel_utils import get_anchored_completion_graph, V_JSON
graph = get_anchored_completion_graph(prompt, completion, version=V_JSON)
plot_graphs([graph], json_labels=True)

Last but not least, we test the isomorphism test for our test samples.
Our test includes two examples. The first one is a correct generated completion, the second example is incorrect.

In [ ]:
import pandas as pd
from mining.isograph import IsoGraph
from mining.plot_utils import to_latex

FILE_PATH = "./test_files/completion_result.jsonl"
data = pd.read_json(FILE_PATH, lines=True)

for idx, example in data.iterrows():
    prompt, completion_gt, full_graph = get_last_graph(example)
    generated_completion = 'e' + example['completion_string']
    graph_gt = get_anchored_completion_graph(prompt, completion_gt, version=V_JSON)
    graph_generated = get_anchored_completion_graph(prompt, generated_completion, version=V_JSON)
    graph_gt = IsoGraph(graph_gt)
    to_latex(graph_gt, None)
    graph_generated = IsoGraph(graph_generated)
    isomorphic_completion = graph_gt == graph_generated
    print(f"Isomorphic??: \t {isomorphic_completion}")

